In [115]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [27]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [28]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [29]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [30]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson', encoding='utf-8') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]
crp_df

,type,properties.fid,properties.구역명,geometry.type,geometry.coordinates,geometry,centroid_polygon_geometry
0,Feature,5,도시재생구역도,Polygon,"[[[127.4833417, 36.6318644], [127.4833417, 36....","POLYGON ((127.4833417 36.6318644, 127.4833417 ...",POLYGON ((127.50347965416363 36.63603755594320...


### 격자(매핑용)

In [37]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [38]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

### 청주시 폐공장현황

In [62]:
factory_df = pd.read_csv('SBJ_2309_001/9.청주시_폐공장현황.csv')
factory_df

,addr,reg_dt,site_area,mnfac_area,addfac_area,cncl_dt,cncl_resn,lon,lat
0,충청북도 청주시 서원구 현도면 시목외천로 381,20200527,1029.00,409.830,83.71,20210702,이전에의한폐업,127.414516,36.522164
1,"충청북도 청주시 청원구 율천남로 92, 1층 (내덕동)",20200226,273.00,132.090,0.00,20230222,이전에의한폐업,127.481527,36.662701
2,충청북도 청주시 서원구 현도면 죽암도원로 12-28,20191025,305.22,155.200,150.02,20210913,이전에의한폐업,127.432720,36.520385
3,충청북도 청주시 청원구 북이면 충청대로 1257-3,20181024,0.00,120.000,0.00,20220624,자진폐업,127.548659,36.752720
4,충청북도 청주시 청원구 북이면 용계내추로 22,20140117,1634.00,683.400,97.50,20230313,이전에의한폐업,127.518171,36.749901
...,...,...,...,...,...,...,...,...,...
204,"충청북도 청주시 서원구 남이면 저산척북로 652, 주 2동",20170306,0.00,187.310,0.00,20210706,이전에의한폐업,127.423037,36.573530
205,충청북도 청주시 청원구 오창읍 각리1길 49,20180130,4286.10,2374.785,395.74,20211126,자진폐업,127.433611,36.707257
206,충청북도 청주시 흥덕구 오송읍 쌍청리 52-11번지,20130204,0.00,196.020,0.00,20210127,자진폐업,127.351052,36.640529
207,충청북도 청주시 흥덕구 옥산면 호죽화산로 140-8,20060608,990.00,199.920,0.00,20220517,이전에의한폐업,127.377697,36.704743


In [64]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
factory_df = geo_transform(factory_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_factory = factory_df[factory_df['geometry'].within(polygon)]

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [66]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13)
# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_factory.iterrows():
    marker = folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 문화재 현황 (폴리곤 영역확장 변경으로 교통분석 후 문화재 필터링 해야할 듯)

https://m.blog.naver.com/zukyun59/222997801975 - 역사문화전경 보존지역 관련 글

In [73]:
culture_assets_df = pd.read_csv('SBJ_2309_001/5.청주시_유적지_문화재현황.csv')
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
culture_assets_df = geo_transform(culture_assets_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_culture = culture_assets_df[culture_assets_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



In [74]:
filtered_points_culture

,gbn,clt_nm,addr,lon,lat,geometry
0,국보,청주 용두사지 철당간,충청북도 청주시 상당구 남문로2가 48-19,127.489758,36.633095,POINT (127.48976 36.63309)
1,충북도 유형문화재,청주 충청도병마절도사영문,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
2,충북도 유형문화재,청주 청녕각,충청북도 청주시 상당구 북문로1가 171-3,127.488321,36.634704,POINT (127.48832 36.63470)
3,충북도 유형문화재,청주 망선루,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
4,충북도 유형문화재,청주 조헌 전장기적비,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
5,충북도 기념물,청주 압각수,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
6,충북도 기념물,청주 척화비,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
7,국가등록문화재,충북도청 본관,충청북도 청주시 상당구 문화동 89,127.491446,36.635428,POINT (127.49145 36.63543)
8,국가등록문화재,주성교육박물관(구 청주공립보통학교 강당),충청북도 청주시 상당구 영동 48-1,127.485226,36.641100,POINT (127.48523 36.64110)
9,국가등록문화재,구 충북산업장려관,충청북도 청주시 상당구 문화동 89-4,127.491058,36.634722,POINT (127.49106 36.63472)


In [75]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_culture.iterrows():
    popup_text = f"{row['gbn']} - {row['clt_nm']}건"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m).add_child(folium.Popup(popup_text, max_width=100))

# 지도 저장
m

### 청주시 학교 현황

In [ ]:
school_df = pd.read_csv('SBJ_2309_001/24.청주시_학교현황.csv')
school_df

,school_nm,estab_gbn,school_chtr,addr,lon,lat
0,금천고등학교,공립,일반고등학교,충청북도 청주시 상당구 호미로233번길 65 금천고등학교 (금천동),127.511943,36.627797
1,대성여자상업고등학교,사립,특성화고등학교,충청북도 청주시 상당구 대성로 220(수동),127.492313,36.645357
2,충북에너지고등학교,공립,특수목적고등학교,충청북도 청주시 상당구 미원면 미원초정로 27(미원면),127.657148,36.639802
3,상당고등학교,공립,일반고등학교,충청북도 청주시 상당구 월평로238번길 3-10(용암동),127.508949,36.606181
4,세광고등학교,사립,일반고등학교,충청북도 청주시 서원구 2순환로 1572(미평동 세광고등학교),127.453040,36.603148
...,...,...,...,...,...,...
171,청주소로초등학교,공립,NaN,충청북도 청주시 흥덕구 옥산면 오송가락로 1020 청주소로초등학교,127.390317,36.672427
172,청주내곡초등학교,공립,NaN,충청북도 청주시 흥덕구 송화로214번길 30 (송절동)청주내곡초등학교,127.457277,36.667208
173,단재초등학교,공립,NaN,충청북도 청주시 상당구 단재로 350 단재초등학교 (방서동),127.500160,36.599530
174,생명초등학교,공립,NaN,충청북도 청주시 청원구 오창읍 중부로 772(오창읍),127.438795,36.711987


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
school_df = geo_transform(school_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_school = school_df[school_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_school

,school_nm,estab_gbn,school_chtr,addr,lon,lat,geometry
0,청주공업고등학교,공립,특성화고등학교,충청북도 청주시 상당구 교서로 17(영동),127.485500,36.638590,POINT (127.48550 36.63859)
1,청주중학교,공립,NaN,충청북도 청주시 상당구 사직대로361번길 125(영동청주중학교),127.485875,36.643274,POINT (127.48587 36.64327)
2,주성초등학교,공립,NaN,충청북도 청주시 상당구 교서로 45 주성초등학교 (영동 주성초등학교),127.485200,36.641424,POINT (127.48520 36.64142)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_school.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 어린이집 유치원 현황

In [ ]:
kinder_df = pd.read_csv('SBJ_2309_001/28.청주시_어린이집_유치원현황.csv')
kinder_df

,fac_nm,fxd_num,prsnt_num,lon,lat
0,행정초등학교병설유치원,17,5,127.531947,36.536620
1,남일초등학교병설유치원,17,10,127.509473,36.575787
2,청남어린이집,89,41,127.512457,36.576130
3,성무어린이집,88,55,127.513181,36.580777
4,경원어린이집,96,72,127.509341,36.586107
...,...,...,...,...,...
674,푸른하늘어린이집,20,18,127.418087,36.711524
675,비봉유치원,139,109,127.413230,36.712757
676,자연어린이집,231,78,127.412753,36.718686
677,세계어린이집,96,66,127.408251,36.727684


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
kinder_df = geo_transform(kinder_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_kinder = kinder_df[kinder_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_kinder

,fac_nm,fxd_num,prsnt_num,lon,lat,geometry
0,센트럴칸타빌어린이집,42,38,127.492188,36.633124,POINT (127.49219 36.63312)
1,주성초등학교병설유치원,17,13,127.485254,36.641427,POINT (127.48525 36.64143)
2,성안나유치원,216,76,127.491409,36.641805,POINT (127.49141 36.64181)
3,은물영아전담어린이집,32,23,127.490870,36.642238,POINT (127.49087 36.64224)
4,코아루휴티스어린이집,34,30,127.489458,36.643841,POINT (127.48946 36.64384)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_kinder.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 인도(보도) 시각화

In [19]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/17.청주시_인도(보도).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
sidewalk_df = pd.json_normalize(geojson_data['features'])
sidewalk_df['geometry'] = sidewalk_df['geometry.coordinates'].apply(make_lin)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
sidewalk_df = gpd.GeoDataFrame(sidewalk_df, geometry='geometry')

In [32]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.015)

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_sidewalk = sidewalk_df[sidewalk_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_sidewalk

,type,properties.UFID,properties.QUAL,properties.BYYN,properties.KIND,geometry.type,geometry.coordinates,geometry
0,Feature,100036706050A0031fb2bb5069e324009,미분류,무,인도,LineString,"[[127.48704805856802, 36.64879715261194], [127...","LINESTRING (127.48705 36.64880, 127.48703 36.6..."
1,Feature,100036706040A0031944d0191c32c411d,미분류,무,인도,LineString,"[[127.4905110148085, 36.65017086233968], [127....","LINESTRING (127.49051 36.65017, 127.49050 36.6..."
2,Feature,100036706050A0031b5e7f9c289f64455,미분류,무,인도,LineString,"[[127.48811663566829, 36.645383249505215], [12...","LINESTRING (127.48812 36.64538, 127.48809 36.6..."
3,Feature,100036706050A00319fde6787305a4ceb,미분류,무,인도,MultiLineString,"[[[127.4821726136577, 36.646733069397726], [12...","LINESTRING (127.48217 36.64673, 127.48202 36.6..."
4,Feature,100036706050A003171957b43200a469d,미분류,무,인도,LineString,"[[127.4892521501864, 36.64684313842911], [127....","LINESTRING (127.48925 36.64684, 127.48907 36.6..."
...,...,...,...,...,...,...,...,...
624,Feature,100036706060A00318412b193c1854672,미분류,유,자전거도로,LineString,"[[127.49352268082293, 36.62464723956152], [127...","LINESTRING (127.49352 36.62465, 127.49340 36.6..."
625,Feature,100036706060A00319220d74c063e442a,블록,유,자전거도로,LineString,"[[127.4940249394998, 36.624428771048855], [127...","LINESTRING (127.49402 36.62443, 127.49389 36.6..."
626,Feature,100036706060A003119b74a925b834b89,블록,유,자전거도로,LineString,"[[127.49415527886981, 36.62422537476052], [127...","LINESTRING (127.49416 36.62423, 127.49405 36.6..."
627,Feature,100036706060A0031ca0403c6c49b4b58,블록,유,자전거도로,LineString,"[[127.4895393709331, 36.62559462233704], [127....","LINESTRING (127.48954 36.62559, 127.48970 36.6..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_sidewalk.iterrows():
    popup_text = f"도로명:{row['properties.KIND']}"
    color = 'black' if row['properties.KIND'] == '인도' else 'blue'
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {'fillColor': color, 'color': color, 'weight': 2}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 상권정보 시각화

In [71]:
trading_area_df = pd.read_csv('SBJ_2309_001/18.청주시_상권정보.csv')
trading_area_df = geo_transform(trading_area_df)
trading_area_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311259000,성화.개신.죽림동,"충청북도 청주시 서원구 복대로 7, (개신동)",127.446061,36.617928,POINT (127.44606 36.61793)
1,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,4311154500,성안동,"충청북도 청주시 상당구 대성로 38, (서운동)",127.493371,36.628602,POINT (127.49337 36.62860)
2,D,소매,D07,가정/주방/인테리어,D07A15,유리/페인트/철물건축자재,G47519,"페인트, 유리 및 기타 건설자재 소매업",4311257000,수곡1동,"충청북도 청주시 서원구 매봉로 64, (수곡동, 산남주공3차)",127.478662,36.616083,POINT (127.47866 36.61608)
3,D,소매,D15,가구소매,D15A01,일반가구소매,G47520,가구 소매업,4311231000,남이면,"충청북도 청주시 서원구 남이면 청남로 1272, (척산리)",127.440092,36.562169,POINT (127.44009 36.56217)
4,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311454000,율량.사천동,"충청북도 청주시 청원구 공항로 118, (율량동)",127.485764,36.667772,POINT (127.48576 36.66777)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46118,L,부동산,L01,부동산중개,L01A01,부동산중개,L68221,부동산 자문 및 중개업,4311454000,율량.사천동,"충청북도 청주시 청원구 사뜸로61번길 106, (사천동, 대창5차)",127.476366,36.664730,POINT (127.47637 36.66473)
46119,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311425300,오창읍,"충청북도 청주시 청원구 오창읍 두릉유리로 1130-15, (장대리)",127.455312,36.741021,POINT (127.45531 36.74102)
46120,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 무심동로390번길 15, (서문동)",127.484757,36.634341,POINT (127.48476 36.63434)
46121,D,소매,D02,선물/팬시/기념품,D02A02,꽃집/꽃배달,G47851,화초 및 산식물 소매업,4311374700,가경동,"충청북도 청주시 흥덕구 풍년로198번길 56-1, (가경동)",127.432312,36.631408,POINT (127.43231 36.63141)


['음식', '소매', '생활서비스', '학문/교육', '부동산', '관광/여가/오락', '숙박', '스포츠'],

In [ ]:
# a = filtered_trading_area[filtered_trading_area['com_lc_nm'] == '소매']
# a['com_mc_nm'].unique()

In [ ]:
# filtered_trading_area['com_lc_nm'].value_counts()

In [ ]:
# a['com_mc_nm'].value_counts()

In [72]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_trading_area = trading_area_df[trading_area_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_trading_area

,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,4311154500,성안동,"충청북도 청주시 상당구 남사로140번길 6, (남문로2가)",127.490409,36.631851,POINT (127.49041 36.63185)
1,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,건강보조식품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로3번길 18-1, (석교동)",127.490050,36.628551,POINT (127.49005 36.62855)
2,D,소매,D05,의복의류,D05A10,한복/갈옷/민속옷,G47412,한복 소매업,4311154500,성안동,"충청북도 청주시 상당구 무심동로336번길 100-2, (남문로1가)",127.488456,36.631302,POINT (127.48846 36.63130)
3,D,소매,D05,의복의류,D05A02,캐쥬얼/스포츠의류,G47416,셔츠 및 기타 의복 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로 55, (남문로2가)",127.490343,36.633284,POINT (127.49034 36.63328)
4,D,소매,D07,가정/주방/인테리어,D07A02,주방용품,G47592,식탁 및 주방용품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로1번길 22, (석교동)",127.489694,36.628361,POINT (127.48969 36.62836)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,D,소매,D23,자동차/자동차용품,D23A11,자동차유리전문점,G45219,기타 자동차신품 부품 및 내장품 판매업,4311152500,중앙동,"충청북도 청주시 상당구 사북로 138, (영동)",127.486152,36.644819,POINT (127.48615 36.64482)
3174,F,생활서비스,F05,인력/고용/용역알선,F05A02,인력공급/고용알선,NaN,NaN,4311154500,성안동,"충청북도 청주시 상당구 용담로 7, (문화동, 문화동센트럴칸타빌)",127.492115,36.632598,POINT (127.49211 36.63260)
3175,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 사직대로362번길 46, (서문동)",127.487471,36.634714,POINT (127.48747 36.63471)
3176,D,소매,D16,화장품소매,D16A01,화장품판매점,G47813,화장품 및 방향제 소매업,4311152500,중앙동,"충청북도 청주시 상당구 상당로 127, (북문로2가)",127.489698,36.639536,POINT (127.48970 36.63954)


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_trading_area.iterrows():
    popup_text = f"{row['com_lc_nm']}-{row['com_mc_nm']}-{row['com_sc_nm']}"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m